# EEG-GAN

## Initialization

### Imports

In [ ]:
import os, requests
from matplotlib import rcParams
from matplotlib import pyplot as plt
import numpy as np
from scipy import signal
import pandas as pd
import torch
import torch.nn as nn
from torch.utils import data
from Datasets import process
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()

### Seeds and Init

In [ ]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")
torch.backends.cudnn.benchmark = True

np.random.seed(1)
torch.manual_seed(1)
torch.cuda.manual_seed(1)
torch.use_deterministic_algorithms(True)

## Loading the Dataset

In [ ]:
# This is the name we'll set for the file
fname = 'motor_imagery.npz'
# Options are 'test' and 'real'. Test is smaller version of the dataset, real is the full dataset.
mode = 'test'
# 12 is the hand, 11 is the tongue
desired_stim = 12
# Load the data
DataLoad = np.load(fname, allow_pickle=True)['dat']
# Print the data
type(DataLoad), len(DataLoad), DataLoad.shape, DataLoad[0][0].keys()

In [ ]:
# This is where we'll keep all the processed data
realV = {}
imagV = {}

# This is where we'll hold all the metadata
realMeta = {}
imagineMeta = {}

# These are the keys we'll be filtering for in the dataset
desiredKeys = ['t_off', 'stim_id', 't_on', 'V', 'scale_uv', 'locs', 'srate']

if mode == 'test':
    length_of_data = 1
elif mode == 'real':
    length_of_data = len(DataLoad)

trange = np.arange(0, 2000)

In [ ]:
for i in range(length_of_data):
    print(f"Sample rate of participant (real) {i}: {DataLoad[i][0]['srate']}")
    print(f"Sample rate of participant (imagine) {i}: {DataLoad[i][1]['srate']}")
    
    x = process.preprocess(DataLoad[i][0])
    nt, nchan = x.shape
    nstim = len(DataLoad[i][0]['t_on'])
    ts = DataLoad[i][0]['t_on'][:, np.newaxis] + trange
    V_epochs = np.reshape(x[ts, :], (nstim, 2000, nchan))
    V_epochs = V_epochs[DataLoad[i][0]['stim_id']==desired_stim]
    print(V_epochs.shape)
    realV[i] = V_epochs
    realMeta[i] = {key: DataLoad[i][0][key] for key in desiredKeys}

    y = process.preprocess(DataLoad[i][1])
    nt, nchan = y.shape
    nstim = len(DataLoad[i][1]['t_on'])
    ts = DataLoad[i][1]['t_on'][:, np.newaxis] + trange
    V_epochs = np.reshape(y[ts, :], (nstim, 2000, nchan))
    V_epochs = V_epochs[DataLoad[i][1]['stim_id']==desired_stim]
    print(V_epochs.shape)
    imagV[i] = V_epochs
    imagineMeta[i] = {key: DataLoad[i][1][key] for key in desiredKeys}

In [ ]:
class EEGDataset(torch.utils.data.Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        return self.data[index]

In [ ]:
EEG = EEGDataset(realV[0])
plt.plot(trange, EEG.__getitem__(1))
plt.show()

## GAN